In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import *

In [0]:
df=spark.read.load("/FileStore/tables/googleplaystore.csv",format="csv",sep=",",header="true",escape='"',inferSchema="true")

In [0]:
df.count()

Out[3]: 10841

In [0]:
df.show(5)

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|    Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design| January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|  August 1, 2018|             1.2.4|4.0.3 and up|
|Ske

In [0]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [0]:
df=df.drop("Size","Content Rating","Last Updated","Android Ver")

In [0]:
df.show(5)

+--------------------+--------------+------+-------+-----------+----+-----+--------------------+------------------+
|                 App|      Category|Rating|Reviews|   Installs|Type|Price|              Genres|       Current Ver|
+--------------------+--------------+------+-------+-----------+----+-----+--------------------+------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|    10,000+|Free|    0|        Art & Design|             1.0.0|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|   500,000+|Free|    0|Art & Design;Pret...|             2.0.0|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5,000,000+|Free|    0|        Art & Design|             1.2.4|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50,000,000+|Free|    0|        Art & Design|Varies with device|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|   100,000+|Free|    0|Art & Design;Crea...|               1.1|
+--------------------+--------------+------+-------+-----------+----+---

In [0]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Current Ver: string (nullable = true)



In [0]:
df=df.withColumn("Rating",df["rating"].cast("float")) \
.withColumn("Reviews",col("Reviews").cast("int")) \
    .withColumn("Installs",regexp_replace("Installs",r"[^a-zA-Z0-9\s]",""))\
        .withColumn("Installs",col("Installs").cast("int"))\
            .withColumn("Price",col("Price").cast("int"))
    
    


In [0]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Current Ver: string (nullable = true)



In [0]:
df.show(5)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|       Current Ver|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|             1.0.0|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|             2.0.0|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|    0|        Art & Design|             1.2.4|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|    0|        Art & Design|Varies with device|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|    0|Art & Design;Crea...|               1.1|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+---

In [0]:
df.createOrReplaceTempView("store")

In [0]:
%sql select App,sum(Reviews) as Total_Reviews from store
group by App
order by sum(Reviews) desc
LIMIT 10

App,Total_Reviews
Instagram,266241989
WhatsApp Messenger,207348304
Clash of Clans,179558781
Messenger – Text and Video Chat for Free,169932272
Subway Surfers,166331958
Candy Crush Saga,156993136
Facebook,156286514
8 Ball Pool,99386198
Clash Royale,92530298
Snapchat,68045010


In [0]:
%sql select App,sum(Installs) as Total_Installs
from store
group by App
order by sum(Installs) desc
LIMIT 10


App,Total_Installs
Subway Surfers,6000000000
Instagram,4000000000
Hangouts,4000000000
Google Drive,4000000000
Google News,4000000000
Google Photos,4000000000
Candy Crush Saga,3500000000
WhatsApp Messenger,3000000000
Messenger – Text and Video Chat for Free,3000000000
Google Chrome: Fast & Secure,3000000000


In [0]:
%sql select Category,sum(Installs)
from store
group by Category
order by sum(Installs) desc
LIMIT 10

Category,sum(Installs)
GAME,35086024415
COMMUNICATION,32647276251
PRODUCTIVITY,14176091369
SOCIAL,14069867902
TOOLS,11452771915
FAMILY,10258263505
PHOTOGRAPHY,10088247655
NEWS_AND_MAGAZINES,7496317760
TRAVEL_AND_LOCAL,6868887146
VIDEO_PLAYERS,6222002720


Databricks visualization. Run in Databricks to view.

In [0]:
%sql select App
from store
where Type='Paid'
group by App
LIMIT 10

App
Ad Blocker for SayHi
Puffin Browser Pro
"Moco+ - Chat, Meet People"
"Private Dating, Hide App- Blue for PrivacyHider"
Sago Mini Hat Maker
AMBW Dating App: Asian Men Black Women Interracial
Truth or Dare Pro
TurboScan: scan documents and receipts in PDF
Tiny Scanner Pro: PDF Doc Scan
Calculator


In [0]:
%sql 
select App,sum(Installs)
from store
group by App, Category
having sum(Installs) > 1000000 and Category="GAME"
order by 2 desc
LIMIT 10

App,sum(Installs)
Subway Surfers,6000000000
Temple Run 2,3000000000
Candy Crush Saga,3000000000
Pou,1500000000
My Talking Tom,1000000000
Sniper 3D Gun Shooter: Free Shooting Games - FPS,600000000
Helix Jump,600000000
slither.io,600000000
8 Ball Pool,600000000
ROBLOX,500000000


Databricks visualization. Run in Databricks to view.